In [31]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [32]:
load_dotenv(override=True)

True

In [33]:
instructions1 = "Eres un agente de ventas que trabaja para ComplAI, \
    una empresa que ofrece una herramienta SaaS para garantizar el cumplimiento de SOC2\
    y prepararse para auditorías, impulsada por IA. Redactas correos electrónicos en frío profesionales y serios."

instructions2 = "Eres un agente de ventas con sentido del humor y atractivo \
    que trabaja para ComplAI, una empresa que ofrece una herramienta SaaS para \
    garantizar el cumplimiento de SOC2 y prepararse para auditorías, impulsada por IA. \
    Redactas correos electrónicos en frío ingeniosos y atractivos que probablemente obtengan respuesta."

instructions3 = "Eres un agente de ventas muy activo que trabaja para ComplAI, \
    una empresa que ofrece una herramienta SaaS para garantizar el cumplimiento de SOC2\
    y prepararse para auditorías, impulsada por IA. Redactas correos electrónicos en frío concisos y directos."

In [34]:
sales_agent1 = Agent(
        name="Agente de ventas profesional",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Agente de ventas atractivo",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Agente de ventas ocpado",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [35]:

result = Runner.run_streamed(sales_agent1, input="Escribe un correo electrónico de ventas en frío")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Asunto: Optimiza tu cumplimiento de SOC 2 con ComplAI

Estimado/a [Nombre del destinatario]:

Espero que este mensaje te encuentre bien. Me presento, soy [Tu Nombre] de ComplAI, y me gustaría compartir cómo nuestra herramienta SaaS puede facilitar el cumplimiento de SOC 2 y la preparación para auditorías de manera eficiente y efectiva.

En un entorno empresarial cada vez más regulado, sabemos que garantizar la seguridad y la privacidad de los datos es crucial. ComplAI utiliza inteligencia artificial para automatizar y agilizar los procesos de cumplimiento, lo que te permite enfocarte en lo que realmente importa: hacer crecer tu negocio.

Algunas de las características clave de nuestra plataforma incluyen:

1. **Automatización de requisitos**: Simplificamos la gestión de evidencias y controles necesarios para cumplir con los estándares SOC 2.
2. **Informes en tiempo real**: Monitorea el estado de cumplimiento en tiempo real y genera informes fácilmente para auditorías.
3. **Asesoramient

#Se procesan los 3 agentes al mismo tiempo, no es multihilo, es un bucle de eventos donde si uno espera procesa otro, de este modo aprovechando el tiempo de espera de un proceso para trabajar en otro

In [36]:
message = "Escribe un correo electrónico de ventas en frío"

with trace("Correos electrónicos fríos en paralelo"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


Asunto: Acelere el Cumplimiento de SOC 2 con ComplAI

Estimado/a [Nombre del destinatario],

Espero que este mensaje le encuentre bien. Mi nombre es [Tu Nombre] y soy representante de ComplAI, una empresa que ofrece soluciones innovadoras para la gestión del cumplimiento normativo.

Entendemos que la auditoría SOC 2 puede ser un proceso complejo y demandante. Por ello, hemos desarrollado una herramienta SaaS impulsada por inteligencia artificial, diseñada específicamente para ayudar a organizaciones como la suya a simplificar y agilizar este proceso. 

Con ComplAI, podrá:

- **Automatizar la recopilación de datos** necesaria para el cumplimiento.
- **Identificar y mitigar riesgos** de manera proactiva.
- **Prepararse eficazmente** para auditorías, ahorrando tiempo y recursos.

Me gustaría coordinar una breve llamada para discutir cómo ComplAI puede adaptarse a las necesidades específicas de su organización y ayudarles a alcanzar sus objetivos de cumplimiento.

Quedo a la espera de su r

In [37]:
sales_picker = Agent(
    name="sales_picker",
    instructions="Elige el mejor correo electrónico de ventas en frío entre las opciones disponibles. \
        Imagina que eres un cliente y elige el que probablemente te responda. \
        No des explicaciones; responde solo con el correo electrónico seleccionado.",
    model="gpt-4o-mini"
)

In [38]:
message = "Escribe un correo electrónico de ventas en frío"

with trace("Selección del equipo de ventas"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Emails de ventas en frío:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"El mejor email de ventas:\n{best.final_output}")


El mejor email de ventas:
Asunto: ¡Haz que el cumplimiento no sea un dolor de cabeza! 🎉

Hola [Nombre],

Espero que estés teniendo un día tan brillante como la pantalla de tu computadora después de un espresso. ☕️

Soy [Tu Nombre] de ComplAI, y me gustaría hablarte de cómo podrías transformar la forma en que enfrentas el cumplimiento de SOC2. Sí, sé que “cumplimiento” y “diversión” no suelen estar en la misma oración, pero aquí estamos para cambiar eso.

Imagina tener una herramienta impulsada por IA que no solo te ayude a preparar auditorías, sino que también lo haga sin que te arranques los pelos de la cabeza. ¿Suena tentador? 🙌

Con ComplAI, podrás:
- Automatizar procesos tediosos (adiós a las noches en vela).
- Tener un camino claro hacia el cumplimiento (sin rodeos ni laberintos).
- Prepararte para auditorías como un campeón.

¿Te gustaría tener una charla de 15 minutos para ver cómo podríamos hacer que el cumplimiento sea tan placentero como encontrar dinero en el abrigo? 

Esper

Ahora añadiremos una herramienta.

todo el código JSON repetitivo y la función `handle_tool_calls()` con la lógica `if`.

In [39]:
sales_agent1 = Agent(
        name="Agente de ventas profesional",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Agente de ventas atractivo",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Agente de ventas ocupado",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [40]:
sales_agent3

Agent(name='Agente de ventas ocupado', instructions='Eres un agente de ventas muy activo que trabaja para ComplAI,     una empresa que ofrece una herramienta SaaS para garantizar el cumplimiento de SOC2    y prepararse para auditorías, impulsada por IA. Redactas correos electrónicos en frío concisos y directos.', handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

In [41]:
@function_tool
def send_email(body: str):
    """ Envía un correo electrónico con el cuerpo indicado a todos los clientes potenciales de ventas. """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email()  # Cambiar a tu remitente verificado
    to_email = To("francisco.sdv.mtz@hotmail.com")  # Cambiar a sureceptor
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Email de ventas", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [42]:
# Echémosle un vistazo
send_email

FunctionTool(name='send_email', description='Envía un correo electrónico con el cuerpo indicado a todos los clientes potenciales de ventas.', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001912CF9FB00>, strict_json_schema=True)

#Convertir un agente en un Herramient

In [43]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Escribe un email de ventas en frío")
tool1

FunctionTool(name='sales_agent1', description='Escribe un email de ventas en frío', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001912D5BC540>, strict_json_schema=True)

# Ahora podemos reunir todas las herramientas:

Una herramienta para cada uno de nuestros tres agentes de redacción de correos electrónicos

Y una herramienta para nuestra función de envío de correos electrónicos

In [44]:
description = "Escribe un correo electrónico de ventas en frío"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Escribe un correo electrónico de ventas en frío', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001912D5BC040>, strict_json_schema=True),
 FunctionTool(name='sales_agent2', description='Escribe un correo electrónico de ventas en frío', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000001912D5BCC20>, strict_json_schema=True),
 FunctionTool(name='sales_agent3', description='Escribe un correo electrónico de ventas en frío', params_json_schema={'properties': {'input': 

# Y ahora es el momento para nuestro Gerente de Ventas - nuestro agente de planificación

In [45]:
instructions ="Eres gerente de ventas y trabajas para ComplAI. \
Utilizas las herramientas que te proporcionamos para generar correos electrónicos de ventas en frío. \
Nunca generas correos electrónicos de ventas tú mismo; siempre usas las herramientas. \
Pruebas las tres herramientas de sales_agent una vez antes de elegir la mejor. \
Eliges el mejor correo electrónico y usas la herramienta send_email para enviar el mejor correo electrónico (y solo el mejor) al usuario."


sales_manager = Agent(name="Manager de ventas", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Envía un correo electrónico de ventas frío dirigido a 'Estimado director ejecutivo'"

with trace("Manager de ventas"):
    result = await Runner.run(sales_manager, message)